In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/Colab Notebooks/Lab2/

/content/drive/MyDrive/Colab Notebooks/Lab2


In [3]:
!pip install catboost

In [4]:
import pandas as pd
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import roc_auc_score

In [5]:
names = np.arange(433)

In [6]:
df = pd.read_csv('train.csv', sep=";", names=names)

In [7]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432
0,-0.004125,0.254095,0.001426,-0.010370,-0.538509,5.95534,1.040630,-1.374370,-0.109370,0.107630,0.093296,-0.065212,0.172694,0.001901,231.814,16.2733,230.818,231.0,0.374904,2.26279,265,205,218.25,242.75,231.263,230.721,13.4375,246.442,0.715107,0.196952,0.737071,0.740630,-2.421450,11.93200,1.040630,-0.188370,0.672130,0.804630,0.697440,0.776332,...,9.03663,31.4247,1015.35,1.160000e+09,0.049631,3.68860,26.8278,12.09630,2.47663,2290000.0,4900000.0,1150000.0,3.330000e+07,14.7410,8.34680,31.4249,1015.36,1.160000e+09,0.018150,3.06333,24.5346,9.69082,1.079070,2290000.0,4900000.0,708396.0,20600000.0,10.43340,5.83548,31.4249,1015.36,7.170000e+08,0.027384,2.53425,17.3882,8.05589,1.802470,1410000.0,3030000.0,1
1,0.056847,1.579030,0.112433,0.110630,-0.231725,3.20953,4.610630,-3.848370,-0.799370,1.053630,0.744039,-0.221476,1.212540,-0.001468,110.146,26.8170,105.284,99.0,1.134260,3.29691,185,83,89.00,124.00,107.319,104.875,21.7634,248.732,0.253252,1.820000,0.295189,0.302130,-0.120011,2.77363,4.596630,-3.848370,-1.031370,1.546630,0.931679,0.851883,...,245.66100,31.4447,1016.33,3.870000e+10,1.484670,94.18340,742.0950,330.30600,57.57300,76100000.0,6090000.0,38300000.0,1.110000e+09,436.1270,245.43700,31.4443,1016.34,3.870000e+10,1.595210,93.57250,724.1740,324.99700,55.024000,76200000.0,6090000.0,23700000.0,686000000.0,275.56700,153.79200,31.4443,1016.34,2.390000e+10,0.977498,59.19060,457.2650,206.93100,36.782000,47100000.0,3760000.0,3
2,0.018532,1.517600,-0.005668,-0.016370,0.140371,3.03585,4.608630,-3.848370,-0.991370,0.992630,0.793278,-0.548854,1.201470,0.000902,113.414,23.6464,110.523,111.0,1.142490,4.60444,192,83,95.25,124.75,111.208,109.197,18.2307,246.707,0.361687,1.861510,0.318493,0.074130,0.146642,2.52676,4.596630,-3.801370,-0.971370,1.580130,1.023800,-10.342300,...,213.17900,31.4273,1015.49,2.890000e+10,0.116626,76.05260,629.8800,246.81700,17.09270,57000000.0,6450000.0,28700000.0,8.310000e+08,373.6030,213.18900,31.4266,1015.46,2.900000e+10,0.060533,75.62170,622.3350,235.22500,9.630820,57100000.0,6450000.0,17700000.0,513000000.0,235.92100,134.23000,31.4266,1015.46,1.790000e+10,0.069618,47.55150,388.4170,156.35800,12.402900,35200000.0,3980000.0,3
3,0.027607,1.172440,-0.010288,-0.004370,0.258784,3.24015,4.381630,-3.848370,-0.787370,0.747630,0.602126,-5.609700,0.921838,0.000169,120.000,37.9460,114.532,114.0,3.502160,20.90510,357,83,97.00,135.00,115.943,112.859,23.1220,244.762,0.418786,1.495170,0.344455,0.371630,0.293875,2.59283,4.352630,-2.663370,-0.790870,1.246880,0.924219,-2.964710,...,171.92200,31.4329,1015.63,2.400000e+10,0.208158,63.69630,509.5670,199.37500,17.90690,47300000.0,6730000.0,23700000.0,6.900000e+08,300.0550,171.32600,31.4327,1015.63,2.400000e+10,0.086477,60.61930,499.1940,187.32500,10.357400,47400000.0,6730000.0,14700000.0,426000000.0,190.43900,107.40500,31.4327,1015.63,1.480000e+10,0.101627,38.18200,314.9020,126.08500,12.621500,29200000.0,4160000.0,3
4,-0.068802,1.268170,-0.110191,-0.120370,0.263951,3.45992,4.606630,-3.848370,-0.896370,0.637630,0.642629,-1.178520,0.982575,0.000407,120.268,44.9087,110.484,108.0,3.645720,18.25520,357,83,102.00,119.00,115.282,112.035,25.2481,242.943,0.021666,1.457050,-0.016974,-0.045370,0.229855,3.37388,4.352630,-3.584370,-0.808620,0.899880,0.746768,0.967325,...,164.91500,31.4344,1015.71,2.290000e+10,0.053455,60.61590,490.5950,189.13100,13.51130,45100000.0,6970000.0,22600000.0,6.570000e+08,286.5850,163.88900,31.4347,1015.72,2.290000e+10,0.029538,58.24600,477.7170,175.45200,5.849970,45100000.0,6970000.0,14000000.0,406000000.0,182.16400,103.63800,31.4347,1015.72,1.410000e+10,0.069863,37.56610,300.9710,120.44100,10.731000,27900000.0,4310000.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [8]:
y_df = np.array(df.loc[:, 432])
y_df

array([1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [9]:
x_df = df.loc[:, 0:431]
x_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431
0,-0.004125,0.254095,0.001426,-0.010370,-0.538509,5.95534,1.040630,-1.374370,-0.109370,0.107630,0.093296,-0.065212,0.172694,0.001901,231.814,16.2733,230.818,231.0,0.374904,2.26279,265,205,218.25,242.75,231.263,230.721,13.4375,246.442,0.715107,0.196952,0.737071,0.740630,-2.421450,11.93200,1.040630,-0.188370,0.672130,0.804630,0.697440,0.776332,...,16.2883,9.03663,31.4247,1015.35,1.160000e+09,0.049631,3.68860,26.8278,12.09630,2.47663,2290000.0,4900000.0,1150000.0,3.330000e+07,14.7410,8.34680,31.4249,1015.36,1.160000e+09,0.018150,3.06333,24.5346,9.69082,1.079070,2290000.0,4900000.0,708396.0,20600000.0,10.43340,5.83548,31.4249,1015.36,7.170000e+08,0.027384,2.53425,17.3882,8.05589,1.802470,1410000.0,3030000.0
1,0.056847,1.579030,0.112433,0.110630,-0.231725,3.20953,4.610630,-3.848370,-0.799370,1.053630,0.744039,-0.221476,1.212540,-0.001468,110.146,26.8170,105.284,99.0,1.134260,3.29691,185,83,89.00,124.00,107.319,104.875,21.7634,248.732,0.253252,1.820000,0.295189,0.302130,-0.120011,2.77363,4.596630,-3.848370,-1.031370,1.546630,0.931679,0.851883,...,440.5850,245.66100,31.4447,1016.33,3.870000e+10,1.484670,94.18340,742.0950,330.30600,57.57300,76100000.0,6090000.0,38300000.0,1.110000e+09,436.1270,245.43700,31.4443,1016.34,3.870000e+10,1.595210,93.57250,724.1740,324.99700,55.024000,76200000.0,6090000.0,23700000.0,686000000.0,275.56700,153.79200,31.4443,1016.34,2.390000e+10,0.977498,59.19060,457.2650,206.93100,36.782000,47100000.0,3760000.0
2,0.018532,1.517600,-0.005668,-0.016370,0.140371,3.03585,4.608630,-3.848370,-0.991370,0.992630,0.793278,-0.548854,1.201470,0.000902,113.414,23.6464,110.523,111.0,1.142490,4.60444,192,83,95.25,124.75,111.208,109.197,18.2307,246.707,0.361687,1.861510,0.318493,0.074130,0.146642,2.52676,4.596630,-3.801370,-0.971370,1.580130,1.023800,-10.342300,...,378.7450,213.17900,31.4273,1015.49,2.890000e+10,0.116626,76.05260,629.8800,246.81700,17.09270,57000000.0,6450000.0,28700000.0,8.310000e+08,373.6030,213.18900,31.4266,1015.46,2.900000e+10,0.060533,75.62170,622.3350,235.22500,9.630820,57100000.0,6450000.0,17700000.0,513000000.0,235.92100,134.23000,31.4266,1015.46,1.790000e+10,0.069618,47.55150,388.4170,156.35800,12.402900,35200000.0,3980000.0
3,0.027607,1.172440,-0.010288,-0.004370,0.258784,3.24015,4.381630,-3.848370,-0.787370,0.747630,0.602126,-5.609700,0.921838,0.000169,120.000,37.9460,114.532,114.0,3.502160,20.90510,357,83,97.00,135.00,115.943,112.859,23.1220,244.762,0.418786,1.495170,0.344455,0.371630,0.293875,2.59283,4.352630,-2.663370,-0.790870,1.246880,0.924219,-2.964710,...,306.7190,171.92200,31.4329,1015.63,2.400000e+10,0.208158,63.69630,509.5670,199.37500,17.90690,47300000.0,6730000.0,23700000.0,6.900000e+08,300.0550,171.32600,31.4327,1015.63,2.400000e+10,0.086477,60.61930,499.1940,187.32500,10.357400,47400000.0,6730000.0,14700000.0,426000000.0,190.43900,107.40500,31.4327,1015.63,1.480000e+10,0.101627,38.18200,314.9020,126.08500,12.621500,29200000.0,4160000.0
4,-0.068802,1.268170,-0.110191,-0.120370,0.263951,3.45992,4.606630,-3.848370,-0.896370,0.637630,0.642629,-1.178520,0.982575,0.000407,120.268,44.9087,110.484,108.0,3.645720,18.25520,357,83,102.00,119.00,115.282,112.035,25.2481,242.943,0.021666,1.457050,-0.016974,-0.045370,0.229855,3.37388,4.352630,-3.584370,-0.808620,0.899880,0.746768,0.967325,...,293.1130,164.91500,31.4344,1015.71,2.290000e+10,0.053455,60.61590,490.5950,189.13100,13.51130,45100000.0,6970000.0,22600000.0,6.570000e+08,286.5850,163.88900,31.4347,1015.72,2.290000e+10,0.029538,58.24600,477.7170,175.45200,5.849970,45100000.0,6970000.0,14000000.0,406000000.0,182.16400,103.63800,31.4347,1015.72,1.410000e+10,0.069863,37.56610,300.9710,120.44100,10.731000,27900000.0,4310000.0
...,...,...,...,...,...,...,...,...,...,...,...,.

In [10]:
std_scaler = StandardScaler()
normalized_x_df = pd.DataFrame(std_scaler.fit_transform(x_df), columns=x_df.columns)
normalized_x_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431
0,-0.181334,-0.321030,-0.141086,-0.153774,-0.933598,-0.187718,-0.359967,-0.175853,0.123457,-0.308458,-0.354225,0.024828,-0.336827,0.243624,0.406244,-0.285056,0.396418,0.404205,-0.071288,-0.393670,-0.087352,0.614452,0.396144,0.360407,0.428934,0.446321,-0.214149,0.335771,0.370718,-0.355180,0.357781,0.315838,-2.012556,1.861732,-0.238297,0.229711,0.560537,0.069510,0.127403,0.121015,...,-0.553645,-0.550560,0.047355,0.056838,-0.547907,-0.278194,-0.556946,-0.552905,-0.563618,-0.477779,-0.547801,-0.424114,-0.547744,-0.547819,-0.549963,-0.548077,0.047137,0.056679,-0.547909,-0.241058,-0.550700,-0.549591,-0.555614,-0.370949,-0.547791,-0.424106,-0.547904,-0.547740,-0.555843,-0.552360,0.048089,0.057544,-0.547853,-0.288501,-0.558443,-0.555122,-0.566999,-0.499549,-0.547880,-0.424016
1,-0.055925,1.963054,0.067487,0.036743,-0.717874,-0.491147,2.377291,-2.208534,-0.903407,1.004165,0.761327,0.017166,1.783705,0.205847,-1.483444,0.119619,-1.483813,-1.518500,0.359753,-0.299296,-0.714592,-1.298134,-1.539612,-1.331974,-1.469830,-1.461760,0.247553,0.397864,-0.441046,2.471519,-0.368541,-0.347365,0.029269,-0.444420,2.547617,-2.486686,-1.858486,1.082318,0.563543,0.138993,...,0.051779,0.035289,0.113769,0.128188,0.118492,1.877868,0.079437,0.056626,0.170068,1.559131,0.116894,-0.399078,0.119244,0.117308,0.052466,0.037997,0.111159,0.127557,0.118464,2.411692,0.086385,0.047001,0.181391,1.707112,0.117837,-0.399071,0.120634,0.117648,0.055693,0.040360,0.111777,0.128073,0.118276,1.817563,0.082833,0.051267,0.171427,1.513416,0.118255,-0.399149
2,-0.134731,1.857154,-0.154415,-0.163221,-0.456225,-0.510340,2.375758,-2.208534,-1.189143,0.919525,0.845736,0.001116,1.761130,0.232422,-1.432687,-0.002071,-1.405344,-1.343709,0.364424,-0.179971,-0.659709,-1.298134,-1.446007,-1.321285,-1.410252,-1.396229,0.051651,0.342957,-0.250459,2.543813,-0.330236,-0.692200,0.265843,-0.506584,2.547617,-2.451803,-1.773284,1.128044,0.735068,-2.524681,...,-0.036460,-0.045132,0.055989,0.067031,-0.055474,-0.177538,-0.048063,-0.039001,-0.022430,0.062576,-0.055111,-0.391504,-0.053113,-0.055043,-0.036921,-0.041718,0.052747,0.063912,-0.053721,-0.169767,-0.039969,-0.039838,-0.028444,-0.041520,-0.054176,-0.391497,-0.053831,-0.055349,-0.035751,-0.038006,0.053670,0.064741,-0.054125,-0.194882,-0.048907,-0.043643,-0.016351,0.110473,-0.055240,-0.391655
3,-0.116066,1.262125,-0.163097,-0.144327,-0.372959,-0.487763,2.201708,-2.208534,-0.885548,0.579575,0.518049,-0.247008,1.190883,0.224205,-1.330397,0.546757,-1.345298,-1.300011,1.703866,1.307631,0.633975,-1.298134,-1.419797,-1.175206,-1.337714,-1.340706,0.322892,0.290219,-0.150101,1.905795,-0.287563,-0.242251,0.396467,-0.489947,2.356457,-1.607197,-1.516968,0.673168,0.549653,-0.769172,...,-0.139233,-0.147278,0.074585,0.077224,-0.142458,-0.040016,-0.134956,-0.141528,-0.131815,0.092677,-0.142464,-0.385613,-0.142883,-0.142145,-0.142068,-0.145201,0.072878,0.076207,-0.142476,-0.126127,-0.145569,-0.144842,-0.140407,-0.013530,-0.141534,-0.385606,-0.141064,-0.142348,-0.140656,-0.145468,0.073695,0.076976,-0.143199,-0.123930,-0.154957,-0.144986,-0.128756,0.123053,-0.142717,-0.385523
4,-0.314363,1.427156,-0.350807,-0.326971,-0.369326,-0.463477,2.374224,-2.208534,-1.047763,0.426944,0.587483,-0.029756,1.314743,0.226871,-1.326234,0.813991,-1.405928,-1.387407,1.785356,1.065800,0.633975,-1.298134,-1.344913,-1.403232,-1.347840,-1.353200,0.440792,0.240897,-0.848086,1.839405,-0.881643,-0.872936,0.339669,-0.293272,2.356457,-2.290749,-1.542174,0.199523,0.219249,0.166462,...,-0.158647,-0.164627,0.079566,0.083048,-0.161985,-0.272449,-0.156618,-0.157695,-0.155435,-0.069828,-0.162276,-0.380564,-0.162632,-0.162531,-0.161325,-0.163585,0.079478,0.082716,-0.162002,-0.221902,-0.162275,-0.163156,-0.168159,-0.187165,-0.16224

# KNN



In [11]:
k_to_accuracy = {}

skf = StratifiedKFold(n_splits=5)

for k in range (1, 51):
    clf = KNN(n_neighbors=k)
    k_to_accuracy[k] = cross_val_score(clf, normalized_x_df, y_df, cv=skf).mean()

for k in sorted(k_to_accuracy):
    print('k = %d, accuracy = %f' % (k, k_to_accuracy[k]))

k = 1, accuracy = 0.714286
k = 2, accuracy = 0.737500
k = 3, accuracy = 0.737500
k = 4, accuracy = 0.748214
k = 5, accuracy = 0.732143
k = 6, accuracy = 0.723214
k = 7, accuracy = 0.730357
k = 8, accuracy = 0.735714
k = 9, accuracy = 0.737500
k = 10, accuracy = 0.735714
k = 11, accuracy = 0.737500
k = 12, accuracy = 0.748214
k = 13, accuracy = 0.746429
k = 14, accuracy = 0.757143
k = 15, accuracy = 0.742857
k = 16, accuracy = 0.760714
k = 17, accuracy = 0.755357
k = 18, accuracy = 0.757143
k = 19, accuracy = 0.760714
k = 20, accuracy = 0.751786
k = 21, accuracy = 0.748214
k = 22, accuracy = 0.757143
k = 23, accuracy = 0.748214
k = 24, accuracy = 0.746429
k = 25, accuracy = 0.750000
k = 26, accuracy = 0.757143
k = 27, accuracy = 0.762500
k = 28, accuracy = 0.767857
k = 29, accuracy = 0.762500
k = 30, accuracy = 0.760714
k = 31, accuracy = 0.753571
k = 32, accuracy = 0.760714
k = 33, accuracy = 0.755357
k = 34, accuracy = 0.762500
k = 35, accuracy = 0.751786
k = 36, accuracy = 0.758929
k

Напишем функцию, которая будет возвращать значение k при котором достигается максимум accuracy

In [12]:
def get_best_k(k_to_accuracy):
    max_accuracy = max(list(k_to_accuracy.values()))
    best_k = list(k_to_accuracy.keys())[list(k_to_accuracy.values()).index(max_accuracy)]
    print("Best k = ", best_k)
    return best_k

Лучший параметр k:

In [13]:
get_best_k(k_to_accuracy);

Best k =  28


# Деревья решений




In [14]:
k_to_accuracy = {}

skf = StratifiedKFold(n_splits=5)

for k in range (1, 51):
    clf = DTC(max_depth=k, random_state=1)
    k_to_accuracy[k] = cross_val_score(clf, normalized_x_df, y_df, cv=skf).mean()

for k in sorted(k_to_accuracy):
    print('k = %d, accuracy = %f' % (k, k_to_accuracy[k]))

k = 1, accuracy = 0.664286
k = 2, accuracy = 0.871429
k = 3, accuracy = 0.889286
k = 4, accuracy = 0.851786
k = 5, accuracy = 0.791071
k = 6, accuracy = 0.773214
k = 7, accuracy = 0.773214
k = 8, accuracy = 0.771429
k = 9, accuracy = 0.771429
k = 10, accuracy = 0.771429
k = 11, accuracy = 0.771429
k = 12, accuracy = 0.771429
k = 13, accuracy = 0.771429
k = 14, accuracy = 0.771429
k = 15, accuracy = 0.771429
k = 16, accuracy = 0.771429
k = 17, accuracy = 0.771429
k = 18, accuracy = 0.771429
k = 19, accuracy = 0.771429
k = 20, accuracy = 0.771429
k = 21, accuracy = 0.771429
k = 22, accuracy = 0.771429
k = 23, accuracy = 0.771429
k = 24, accuracy = 0.771429
k = 25, accuracy = 0.771429
k = 26, accuracy = 0.771429
k = 27, accuracy = 0.771429
k = 28, accuracy = 0.771429
k = 29, accuracy = 0.771429
k = 30, accuracy = 0.771429
k = 31, accuracy = 0.771429
k = 32, accuracy = 0.771429
k = 33, accuracy = 0.771429
k = 34, accuracy = 0.771429
k = 35, accuracy = 0.771429
k = 36, accuracy = 0.771429
k

In [15]:
dtc = DTC(max_depth=get_best_k(k_to_accuracy))

Best k =  3


# SVM

In [16]:
k_to_accuracy = {}

skf = StratifiedKFold(n_splits=5)

for k in range (1, 30):
    clf = svm.SVC(kernel="poly", degree=k, random_state=1)
    k_to_accuracy[k] = cross_val_score(clf, normalized_x_df, y_df, cv=skf).mean()

for k in sorted(k_to_accuracy):
    print('k = %d, accuracy = %f' % (k, k_to_accuracy[k]))

k = 1, accuracy = 0.853571
k = 2, accuracy = 0.557143
k = 3, accuracy = 0.535714
k = 4, accuracy = 0.462500
k = 5, accuracy = 0.464286
k = 6, accuracy = 0.446429
k = 7, accuracy = 0.444643
k = 8, accuracy = 0.441071
k = 9, accuracy = 0.433929
k = 10, accuracy = 0.425000
k = 11, accuracy = 0.421429
k = 12, accuracy = 0.417857
k = 13, accuracy = 0.414286
k = 14, accuracy = 0.414286
k = 15, accuracy = 0.414286
k = 16, accuracy = 0.410714
k = 17, accuracy = 0.410714
k = 18, accuracy = 0.410714
k = 19, accuracy = 0.410714
k = 20, accuracy = 0.408929
k = 21, accuracy = 0.408929
k = 22, accuracy = 0.408929
k = 23, accuracy = 0.408929
k = 24, accuracy = 0.408929
k = 25, accuracy = 0.408929
k = 26, accuracy = 0.407143
k = 27, accuracy = 0.407143
k = 28, accuracy = 0.407143
k = 29, accuracy = 0.407143


In [17]:
clf_svm = svm.SVC(kernel="poly", degree=get_best_k(k_to_accuracy))

Best k =  1


Лучший результат показал SVM - похоже, что данные хорошо разделимы. Для KNN, возможно, слишком много признаков. 

# Второе задание
Перейдём ко второму заданию. Необходимо очистить данные и разделить их 50 на 50

In [18]:
train_X_0, test_X_0, train_Y_0, test_Y_0 = train_test_split(normalized_x_df, y_df, test_size=0.5, random_state=1)
normalized_x_df.shape, y_df.shape

((560, 432), (560,))

Функция для нахождения и удаления выбросов:

In [19]:
def fit_lof(X, y):
    X0=X[y==1]
    X1=X[y==2]
    X2=X[y==3]
    lof0 = LocalOutlierFactor(novelty=True).fit(X0)
    lof1 = LocalOutlierFactor(novelty=True).fit(X1)
    lof2 = LocalOutlierFactor(novelty=True).fit(X2)
    return [lof0, lof1, lof2]
    
def remove_outliers(X, y, lofs):
    X0=X[y==1]
    X1=X[y==2]
    X2=X[y==3]
    
    X0_=X0[lofs[0].predict(X0)==1]
    X1_=X1[lofs[1].predict(X1)==1]
    X2_=X2[lofs[2].predict(X2)==1]
    X_ = np.vstack([X0_, X1_, X2_])
    
    y_ = np.hstack([np.ones(X0_.shape[0]), np.ones(X1_.shape[0]) * 2, np.ones(X2_.shape[0]) * 3])
    
    print(f"Number of outliers: {len(y) - len(y_)}")
    return X_, y_

Функция для удаления малоинформативных признаков:

In [20]:
def fit_rfecv(X, y, clf):
    rfecv = RFECV(estimator=clf, step=1, cv=5, scoring='accuracy')
    rfecv.fit(X, y)
    return rfecv

def remove_uniformative_features(X, rfecv):
    X_ = X[:, rfecv.ranking_ == 1]
    print(f"Features before: {X.shape[1]}, features after: {X_.shape[1]}")
    return X_

Сначала убираем выбросы, потом неинформативные признаки:

In [21]:
lofs = fit_lof(train_X_0, train_Y_0)
train_X_1, train_Y_1 = remove_outliers(train_X_0, train_Y_0, lofs)


clf = svm.SVC(kernel="linear")
rfecv = fit_rfecv(train_X_1, train_Y_1, clf)

train_X_1 = remove_uniformative_features(train_X_1, rfecv)
test_X_1 = remove_uniformative_features(test_X_0.values, rfecv)

print("Remove outliers before features:")
print(f"Train info: objects: {train_X_1.shape[0]}, features: {train_X_1.shape[1]}")

Number of outliers: 48
Features before: 432, features after: 26
Features before: 432, features after: 26
Remove outliers before features:
Train info: objects: 232, features: 26


Сначала неинформативные признаки, потом выбросы:

In [22]:
clf = svm.SVC(kernel="linear")
rfecv = fit_rfecv(train_X_0, train_Y_0, clf)
train_X_2 = remove_uniformative_features(train_X_0.values, rfecv)
test_X_2 = remove_uniformative_features(test_X_0.values, rfecv)

lofs = fit_lof(train_X_2, train_Y_0)
train_X_2, train_Y_2 = remove_outliers(train_X_2, train_Y_0, lofs)

print("Remove features before outliers:")
print(f"Train info: objects: {train_X_2.shape[0]}, features: {train_X_2.shape[1]}")
print(f"Test info: objects: {test_X_2.shape[0]}, features: {test_X_2.shape[1]}")

Features before: 432, features after: 24
Features before: 432, features after: 24
Number of outliers: 37
Remove features before outliers:
Train info: objects: 243, features: 24
Test info: objects: 280, features: 24


Видим, что после удаления выбросов неинформативных признаков становится меньше

Используя значения параметров, подобранные в прошлом задании, посмотрим лучший скор на неоцищенных и очищенных данных

# KNN

In [23]:
clf = KNN(n_neighbors=28)
clf.fit(train_X_0, train_Y_0)
print(f"Not cleared: {accuracy_score(test_Y_0, clf.predict(test_X_0))}")
clf = KNN(n_neighbors=28)
clf.fit(train_X_1, train_Y_1)
print(f"Clear outliers before features: {accuracy_score(test_Y_0, clf.predict(test_X_1))}")
clf = KNN(n_neighbors=28)
clf.fit(train_X_2, train_Y_2)
print(f"Clear features before outliers: {accuracy_score(test_Y_0, clf.predict(test_X_2))}")

Not cleared: 0.7678571428571429
Clear outliers before features: 0.8678571428571429
Clear features before outliers: 0.8857142857142857


Попробуем другие значения n_neighbors:

In [24]:
clf = KNN(n_neighbors=20)
clf.fit(train_X_0, train_Y_0)
print(f"Not cleared: {accuracy_score(test_Y_0, clf.predict(test_X_0))}")
clf = KNN(n_neighbors=20)
clf.fit(train_X_1, train_Y_1)
print(f"Clear outliers before features: {accuracy_score(test_Y_0, clf.predict(test_X_1))}")
clf = KNN(n_neighbors=20)
clf.fit(train_X_2, train_Y_2)
print(f"Clear features before outliers: {accuracy_score(test_Y_0, clf.predict(test_X_2))}")
print("---------------")
clf = KNN(n_neighbors=5)
clf.fit(train_X_0, train_Y_0)
print(f"Not cleared: {accuracy_score(test_Y_0, clf.predict(test_X_0))}")
clf = KNN(n_neighbors=5)
clf.fit(train_X_1, train_Y_1)
print(f"Clear outliers before features: {accuracy_score(test_Y_0, clf.predict(test_X_1))}")
clf = KNN(n_neighbors=5)
clf.fit(train_X_2, train_Y_2)
print(f"Clear features before outliers: {accuracy_score(test_Y_0, clf.predict(test_X_2))}")

Not cleared: 0.7678571428571429
Clear outliers before features: 0.8714285714285714
Clear features before outliers: 0.8892857142857142
---------------
Not cleared: 0.8107142857142857
Clear outliers before features: 0.9142857142857143
Clear features before outliers: 0.8964285714285715


В целом KNN лучше работает на очищенных данных, причем сначала должны быть обработаны признаки (зависит от параметров)

# Деревья решений

In [25]:
clf = DTC(max_depth=3, random_state=0)
clf.fit(train_X_0, train_Y_0)
print(f"Not cleared: {accuracy_score(test_Y_0, clf.predict(test_X_0))}")
clf = DTC(max_depth=3, random_state=0)
clf.fit(train_X_1, train_Y_1)
print(f"Clear outliers before features: {accuracy_score(test_Y_0, clf.predict(test_X_1))}")
clf = DTC(max_depth=3, random_state=0)
clf.fit(train_X_2, train_Y_2)
print(f"Clear features before outliers: {accuracy_score(test_Y_0, clf.predict(test_X_2))}")

Not cleared: 0.9392857142857143
Clear outliers before features: 0.9142857142857143
Clear features before outliers: 0.8964285714285715


In [26]:
clf = DTC(max_depth=5, random_state=0)
clf.fit(train_X_0, train_Y_0)
print(f"Not cleared: {accuracy_score(test_Y_0, clf.predict(test_X_0))}")
clf = DTC(max_depth=5, random_state=0)
clf.fit(train_X_1, train_Y_1)
print(f"Clear outliers before features: {accuracy_score(test_Y_0, clf.predict(test_X_1))}")
clf = DTC(max_depth=5, random_state=0)
clf.fit(train_X_2, train_Y_2)
print(f"Clear features before outliers: {accuracy_score(test_Y_0, clf.predict(test_X_2))}")
print("-------------------------")
clf = DTC(max_depth=15, random_state=0)
clf.fit(train_X_0, train_Y_0)
print(f"Not cleared: {accuracy_score(test_Y_0, clf.predict(test_X_0))}")
clf = DTC(max_depth=15, random_state=0)
clf.fit(train_X_1, train_Y_1)
print(f"Clear outliers before features: {accuracy_score(test_Y_0, clf.predict(test_X_1))}")
clf = DTC(max_depth=15, random_state=0)
clf.fit(train_X_2, train_Y_2)
print(f"Clear features before outliers: {accuracy_score(test_Y_0, clf.predict(test_X_2))}")

Not cleared: 0.9464285714285714
Clear outliers before features: 0.9321428571428572
Clear features before outliers: 0.9035714285714286
-------------------------
Not cleared: 0.95
Clear outliers before features: 0.9071428571428571
Clear features before outliers: 0.9071428571428571


В целом DT лучше работает на сырых данных. На очищенных особой разницы от порядка обработки нет

# SVM

In [27]:
clf = svm.SVC(kernel="poly", degree=1, random_state=0)
clf.fit(train_X_0, train_Y_0)
print(f"Not cleared: {accuracy_score(test_Y_0, clf.predict(test_X_0))}")
clf = svm.SVC(kernel="poly", degree=1, random_state=0)
clf.fit(train_X_1, train_Y_1)
print(f"Clear outliers before features: {accuracy_score(test_Y_0, clf.predict(test_X_1))}")
clf = svm.SVC(kernel="poly", degree=1, random_state=0)
clf.fit(train_X_2, train_Y_2)
print(f"Clear features before outliers: {accuracy_score(test_Y_0, clf.predict(test_X_2))}")

Not cleared: 0.9
Clear outliers before features: 0.8857142857142857
Clear features before outliers: 0.8928571428571429


In [28]:
clf = svm.SVC(kernel="poly", degree=3, random_state=0)
clf.fit(train_X_0, train_Y_0)
print(f"Not cleared: {accuracy_score(test_Y_0, clf.predict(test_X_0))}")
clf = svm.SVC(kernel="poly", degree=3, random_state=0)
clf.fit(train_X_1, train_Y_1)
print(f"Clear outliers before features: {accuracy_score(test_Y_0, clf.predict(test_X_1))}")
clf = svm.SVC(kernel="poly", degree=3, random_state=0)
clf.fit(train_X_2, train_Y_2)
print(f"Clear features before outliers: {accuracy_score(test_Y_0, clf.predict(test_X_2))}")
print("---------------------------")
clf = svm.SVC(kernel="poly", degree=9, random_state=0)
clf.fit(train_X_0, train_Y_0)
print(f"Not cleared: {accuracy_score(test_Y_0, clf.predict(test_X_0))}")
clf = svm.SVC(kernel="poly", degree=9, random_state=0)
clf.fit(train_X_1, train_Y_1)
print(f"Clear outliers before features: {accuracy_score(test_Y_0, clf.predict(test_X_1))}")
clf = svm.SVC(kernel="poly", degree=9, random_state=0)
clf.fit(train_X_2, train_Y_2)
print(f"Clear features before outliers: {accuracy_score(test_Y_0, clf.predict(test_X_2))}")

Not cleared: 0.5357142857142857
Clear outliers before features: 0.8571428571428571
Clear features before outliers: 0.8785714285714286
---------------------------
Not cleared: 0.4392857142857143
Clear outliers before features: 0.6357142857142857
Clear features before outliers: 0.6964285714285714


В целом SVM значительно лучше работает на очищенных данных. Зависит от параметров, но в среднем признаки лучше обрабатывать перед выбросами

### Общий вывод по заданию 2



1. На неочищенных данных лучше всего отработал DT. Это неудивительно, учитывая особенности алгоритма
2. Последовательность обработки признаков и выбросов зависит от алгоритма и параметром. В общем случае лучше сначала очистить признаки, чтобы адекватнее выделить выбросы
3. На очищенных данных лучше всего отработал тоже отработал DT. Но стоит заметить, что в этом эксперименте большую роль играет разбиение данных части, так что результат может оказать нерелевантным.